<h3> Extracting protein sequences' features using ProtBert-BFD pretrained-model <h3>

<b>1. Load necessary libraries including huggingface transformers<b>

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import Dataset, load_dataset
import re
import numpy as np
import os
import requests
import pandas as pd
from tqdm import tqdm
import skimage.measure

In [2]:
tqdm.pandas()

<b>2. Load the vocabulary and ProtBert-BFD Model<b>

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False )

In [ ]:
model = AutoModel.from_pretrained("Rostlab/prot_bert_bfd")

<b>3. Load the model into the GPU if avilabile<b>

In [5]:
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer,device=0 ) # device=0 for GPU, -1 for CPU

## Load dataset

In [ ]:
df = pd.read_csv('data/Artificial_proteins.csv')
df['Sequence2'] = df['Sequence'].apply(lambda sequence: " ".join(re.sub(r"[UZOB]", "X", sequence)))

<b>5. Extracting sequences' features <b>

In [15]:
def get_embedding(seq):
    embedding = fe(seq)
    features =  np.array(embedding[0][1:len(seq)+1])
    features = skimage.measure.block_reduce(features, (1024, 1), np.average)
    return np.array(features[0], dtype=float)

In [ ]:
features = df['Sequence2'].progress_apply(get_embedding)

In [ ]:
df_embeddings = pd.concat([df, pd.DataFrame(features.tolist()).add_prefix("f")], axis=1)

df_embeddings = df_embeddings.drop(columns=['Sequence2'])

print(df_embeddings.head(n=10))

In [118]:
df_embeddings.to_csv('data/Artificial_proteins_embeddings.csv', index=False)